In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build

import numpy as np
import exafmm as fmm
import exafmm.laplace as laplace

In [2]:
laplace.__doc__

"exafmm's submodule for Laplace kernel"

### Create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((nsrcs, 3))

# generate random charges for sources
src_charges = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = laplace.init_sources(src_coords, src_charges)

# create a list of target instances
targets = laplace.init_targets(trg_coords)

### Create a LaplaceFMM instance

In [5]:
p = 10
ncrit = 400
depth = 4
fmm = laplace.LaplaceFMM(p, ncrit, depth)

### Setup FMM

Given sources, targets and FMM configurations, `setup()` builds the tree and interaction list and pre-compute invariant matrices.

Set `skip_P2P` to `True` to skip near-field interactions for BEM applications.

In [6]:
skip_P2P = False
tree = laplace.setup(sources, targets, fmm, skip_P2P)

### Evaluate

`evaluate()` triggers the evaluation and returns a $n_{trg} \times 4$ `numpy.ndarray`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [7]:
trg_values = laplace.evaluate(tree, fmm)

P2M                  : 2.0449100e-01
M2M                  : 1.0078900e-01
P2L                  : 6.6000000e-05
M2P                  : 4.4000000e-05
P2P                  : 3.8443000e-02
M2L                  : 7.8807900e-01
L2L                  : 1.0230400e-01
L2P                  : 1.4846400e-01


In [8]:
trg_values[6]

array([ 5458.58112562, -5598.38629587,  5838.7463581 ,  2345.01904127])

### Check Accuracy

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list. It only works when `skip_P2P` is set to `False`.

In [9]:
fmm.verify(tree.leafs)

[3.129192145907721e-09, 2.3583016688134237e-07]

### Update charges of sources and run FMM iteratively

In [10]:
niters = 10

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges = np.random.random(nsrcs)          # generate new random charges
    laplace.update_charges(tree, src_charges)      # update charges
    laplace.clear_values(tree)                     # clear values
    trg_values = laplace.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
P2M                  : 1.8961500e-01
M2M                  : 1.0455700e-01
P2L                  : 6.6000000e-05
M2P                  : 4.2000000e-05
P2P                  : 4.2172000e-02
M2L                  : 7.8405200e-01
L2L                  : 9.5967000e-02
L2P                  : 1.5680500e-01
Error:  [2.9880783588163687e-09, 2.329412098002378e-07]
---------- iteration 1 ----------
P2M                  : 1.4056600e-01
M2M                  : 9.9321000e-02
P2L                  : 6.3000000e-05
M2P                  : 4.2000000e-05
P2P                  : 3.8437000e-02
M2L                  : 7.9684300e-01
L2L                  : 9.6931000e-02
L2P                  : 1.4975100e-01
Error:  [3.096014102210114e-09, 2.4003457180950724e-07]
---------- iteration 2 ----------
P2M                  : 1.4206700e-01
M2M                  : 1.0191400e-01
P2L                  : 6.4000000e-05
M2P                  : 4.1000000e-05
P2P                  : 3.8502000e-02
M2L      